# Face Recognition #5 TAR

- We are using: DB1-Youtube (half database from A to I)

We want here to generate all the data that will be used later to compute the TAR in the notebook 'FAR&Stats'. The encodings of each person have been already generated. In particular, each person is described by a vector that contains 20 encodings generated from 20 random images taken from a single folder, the test will be performed on the remaining ones.


In [1]:
import sys
import os
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/PRIM'  # your new root path

sys.path.append(os.path.join(root_path, 'notebooks')) # for importing from utils.py

!pip install face_recognition

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import argparse
import pickle
import cv2
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
import dlib
import numpy as np
import matplotlib.pyplot as plt
import time
import random
print(dlib.DLIB_USE_CUDA)


True


The function below is used to create the encodings, only 20 images of all the one present in the folder are taken in order to generate them.

In [0]:
def createEncodings(dataset_folder, name, detection='cnn'):

    # grab the paths to the input images in our dataset
    print("[INFO] quantifying faces...")
    imagePathsList = list(paths.list_images(dataset_folder))
    '''
    We're going to take only a limit number of images to create the encodings.
    We'll take a random sample over all the images of a person and in particular
    '''
    num = 20
    if len(imagePathsList )<= 0:
      return

    imagePaths = random.choices(imagePathsList, k = num )

    # initialize the list of known encodings and known names
    knownEncodings = []
    knownNames = []
    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        #print("[INFO] processing image {}/{}".format(i + 1,len(imagePaths)))
        name = imagePath.split(os.path.sep)[-3]

        # load the input image and convert it from RGB (OpenCV ordering)
        # to dlib ordering (RGB)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb,model=detection)

        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)

        # loop over the encodings
        for encoding in encodings:
            # add each encoding + name to our set of known names and
            # encodings
            knownEncodings.append(encoding)
            knownNames.append(name)
    #pdb.set_trace()
    # dump the facial encodings + names to disk
    print("[INFO] serializing encodings..." + name)
    data = {"encodings": knownEncodings, "names": knownNames}
    name = root_path + "/#3_encodings/" + name + '.pkl'
    #print(name)
    f = open(name, "wb")
    f.write(pickle.dumps(data))
    f.close()

In the function below we compare the list of the 20 encodings of a known person with only one encoding of a different person generated from an **input image** in order to see the result of the algorithm. If the two person are considered the same the name is returned otherwise we have an 'unknown'.
The decision is based on the **Euclidian distance** $d$ between the encodings,two person are recognized as the same one iff:<br> 
- $d < tolerance$

In [0]:
def faceRecognition(image, encodings, detection='cnn', show=False,tolerance = 0.6):
    start = time.time()
    # the image is already converted in an rgb format
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # format used by openCV
    data = encodings

    # detect the (x, y)-coordinates of the bounding boxes corresponding
    # to each face in the input image, then compute the facial embeddings
    # for each face
    boxes = face_recognition.face_locations(rgb,model=detection)
    encodings = face_recognition.face_encodings(rgb, boxes)

    # initialize the list of names for each face detected
    names = []

    # loop over the facial embeddings
    for encoding in encodings:
        # attempt to match each face in the input image to our known
        # encodings
        matches = face_recognition.compare_faces(data["encodings"],
            encoding)
        name = "unknown"

        # check to see if we have found a match
        if True in matches:
          # Use the known face with the smallest distance to the new face
          face_distances = face_recognition.face_distance(data["encodings"], encoding)
          best_match_index = np.argmin(face_distances)
          if matches[best_match_index]:
              name = data["names"][best_match_index]

        # update the list of names
        try:
            name = name.split('/')[2]
        except:
            name = name
        names.append(name)

    # loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # draw the predicted face name on the image
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0, 255, 0), 2)
    
    
    stop = time.time()
    #print('Time needed:', stop-start, 'sec')
    # show the output image
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if(show== True):
        print(names)
        plt.imshow(rgb)
        plt.show()
    return names

def imageRead(path, show=True):
    # BGR FORMAT #
    image = cv2.imread(path)
    
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if(show== True):
        plt.imshow(rgb)
        plt.show()
    return image
    
def encodingsRead(path):
    ## PAY ATTENTION:: You need to create the encoding for you dataset using encode_face.py
    data = pickle.loads(open(path, "rb").read())
    return data
    

In [0]:
def faceRecOnFolder(dataset_folder, encodings, correct_person, detection='cnn', min_threshold = 0.75, show=False,tolerance = 0.6):
    imagePaths = list(paths.list_images(dataset_folder))

    analysed_frames = 0
    correct_frames = 0
    negative_frames = 0
    neutrum_frames = 0
    total_frames = len(imagePaths)
    start = time.time()

    if total_frames < 30:
      min_frames = total_frames
    else:
      min_frames = total_frames * 50 / 100
    
    min_threshold = min_threshold
    # grab the paths to the input images in our dataset
    print("[INFO] Recognizing faces...")
    print("[INFO] elements:", total_frames)

    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        #print("[INFO] processing image {}/{}".format(i + 1,len(imagePaths)))
        image = imageRead(imagePath, show=False)
        person_rec = faceRecognition(image, encodings, detection='cnn', show=False,tolerance = tolerance)
        analysed_frames += 1
        
        if(len(person_rec) >= 1):
            person = person_rec[0]
            if(person == correct_person):
                correct_frames += 1
            else:
                if(person == 'unknown'):
                    neutrum_frames += 1
                else:
                    negative_frames += 1
        else:
            neutrum_frames += 1
        
            
        if( analysed_frames > min_frames and correct_frames/analysed_frames > min_threshold):
            print(correct_person, "Recognized after", analysed_frames, 'with accuracy:', correct_frames/analysed_frames)
            print('Not detected:', neutrum_frames/analysed_frames, ". Wrong recognition:", negative_frames/analysed_frames)
            correct_rate = correct_frames/analysed_frames
            neutrum_rate = neutrum_frames/analysed_frames
            negative_rate = negative_frames/analysed_frames
            end = time.time()
            print('Time:', end-start)
            return True, correct_rate, neutrum_rate, negative_rate
        
    #for ends
    correct_rate = correct_frames/total_frames
    neutrum_rate = neutrum_frames/total_frames
    negative_rate = negative_frames/total_frames
    end = time.time()
    print(correct_person, "Recognized after", analysed_frames, 'with accuracy:', correct_frames/total_frames)
    print('Not detected:', neutrum_frames/total_frames, ". Wrong recognition:", negative_frames/total_frames)
    print('Time:', end-start)
    if(correct_rate > min_threshold - 0.05):
        return True, correct_rate, neutrum_rate, negative_rate
    else:
        return False, correct_rate, neutrum_rate, negative_rate
        

In [0]:
def walklevel(some_dir, level=1):
    some_dir = some_dir.rstrip(os.path.sep)
    num_sep = some_dir.count(os.path.sep)
    for root, dirs, files in os.walk(some_dir):
        yield root, dirs, files
        num_sep_this = root.count(os.path.sep)
        if num_sep + level <= num_sep_this:
            del dirs[:]
            

def produceNamesFile(path):
    count = 0
    files = 0
    folders = 0
    f = open("valid.txt", "w")
    #path = "./DB1-Youtube/frame_images_DB"

    for name, dirnames, filenames in walklevel(path):
        if(name != "./DB1-Youtube/frame_images_DB"):
            nfolders = len(dirnames)
            if( nfolders > 1):
                count +=1
                ## we are just using people that have more than one video!
                f.write(name.split("\\")[1] + "\n")
                print(name, dirnames, nfolders)

    print("Number of people useful for the test:", count)
    print("FINE")
    f.close()
    return

In [0]:
test_path = root_path + "/frame_images_mattia/"
encodings_folder = root_path + "/#3_encodings/"

The Test compare the encodings of one person with the encodings of himself generated from a different folder in order to compute later the Tar.


In [0]:

def performTest(min_v=0, max_v=100000, threshold=0.75, log=False,tolerance = 0.6):
    print("-------------------------")
    print("TEST STARTED")
    print("-------------------------")
    
    count = 0
    n_test = 0
    n_correct = 0
  
    valid_persons = os.listdir(test_path)

    log_folder_base = 'test_' + str(threshold) + '_' + str(tolerance) + '/'


    if( log == True):
      log_folder = log_folder_base + 'T/'
      try:
          #os.mkdir(root_path + '/logs#5/' + log_folder_base)
          os.mkdir(root_path + '/logs#5/' + log_folder)
      except FileExistsError:
          print('Already existing folder')
      logfile = open(root_path + '/logs#5/' + log_folder + "log" + str(min_v) + "-" + str(max_v) + ".txt", "w")
        
    for person in valid_persons:
        count = count +1
        if(count < min_v):
            continue
        if(count > max_v):
            break

        #person = person[0:len(person)-1]
        test_folder = test_path + person
        #test_folder = test_folder.replace("/", "\\")
        print(test_folder)
        # create the enconding related to first folder of the person
        folder_list = os.listdir(test_folder)
        first_n_folder = folder_list[0]
        train_folder = test_folder + "/" + first_n_folder
        res = os.path.exists(encodings_folder + person + ".pkl")
        if(res == False):
            #print("Qualcosa è andato storto!!")
            print("Creation encodings on",train_folder)
            #muori = 1/0
            createEncodings(train_folder, person)
        
        # test the encodings of testset
        train_flag = 0
        for n_folder in folder_list:
            if( train_flag == 0):
                train_flag = 1
                continue
            
            n_test += 1
            folder = test_folder + "/" + n_folder
            encoding = encodingsRead(encodings_folder + person + ".pkl")
            v, a, b, c = faceRecOnFolder(folder, encoding, person, min_threshold=threshold, show=True,tolerance = tolerance)
            if(log== True):
                line = person + " " + n_folder + " " + str(v) + " " + str(a) + " " + str(b) + " " + str(c) + "\n"
                logfile.write(line)

            if( v == True):
                n_correct += 1
    
    if(log == True):
        logfile.close()
    print("-------------------------")
    print("TEST ENDED")
    print("-------------------------")
    print('Accuracy on videos tested:', n_correct/n_test)
    return


The aim of our experiment is to compare the encoding of each person multiple times, and every time with a different value of tolerance in order to make at each iteration a more strict comparison.

In [11]:
tol_values = [ 0.6, 0.5, 0.4, 0.3, 0.2, 0.1 ]
thr = 0.70

for tol in tol_values:
        print('********')
        print('TOLERANCE', tol)
        print('********')
        performTest(min_v=0, max_v = 200, log=True, threshold= thr, tolerance= tol)

Streaming output truncated to the last 5000 lines.
[INFO] Recognizing faces...
[INFO] elements: 53
Jaime_Pressly Recognized after 27 with accuracy: 0.9629629629629629
Not detected: 0.037037037037037035 . Wrong recognition: 0.0
Time: 0.9412577152252197
[INFO] Recognizing faces...
[INFO] elements: 62
Jaime_Pressly Recognized after 32 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 0.8296246528625488
gdrive/My Drive/Colab Notebooks/PRIM/frame_images_mattia/Jack_Valenti
[INFO] Recognizing faces...
[INFO] elements: 2041
Jack_Valenti Recognized after 1021 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 27.3446102142334
[INFO] Recognizing faces...
[INFO] elements: 61
Jack_Valenti Recognized after 31 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 1.0195739269256592
gdrive/My Drive/Colab Notebooks/PRIM/frame_images_mattia/Jackie_Chan
[INFO] Recognizing faces...
[INFO] elements: 49
Jackie_Chan Recognized after 25 with accuracy: 1.0
Not